In [3]:
import pandas as pd
import numpy as np

import requests
import aiohttp
import asyncio
import math
import nest_asyncio
nest_asyncio.apply()

from google.cloud import bigquery

In [84]:
datagov_name_and_id_buildings = ("buildings", "d_17f5382f26140b1fdae0ba2ef6239d2f")
datagov_name_and_id_median_rent = ("median_rent", "d_23000a00c52996c55106084ed0339566")
datagov_name_and_id_schools = ("schools", "d_688b934f82c1059ed0a6993d2a829089")
datagov_name_and_id_trans1 = ("transactions1_data", "d_ebc5ab87086db484f88045b47411ebc5")
datagov_name_and_id_trans2 = ("transactions2_data", "d_43f493c6c50d54243cc1eab0df142d6a")
datagov_name_and_id_trans3 = ("transactions3_data", "d_2d5ff9ea31397b66239f245f57751537")
datagov_name_and_id_trans4 = ("transactions4_data", "d_ea9ed51da2787afaf8e51f827c304208")
datagov_name_and_id_trans5 = ("transactions5_data", "d_8b84c4ee58e3cfc0ece0d773c8ca6abc")

town_mapping = {
    'AMK': 'ANG MO KIO', 'BB': 'BUKIT BATOK', 'BD': 'BEDOK', 'BH': 'BISHAN', 'BM': 'BUKIT MERAH',
    'BP': 'BUKIT PANJANG', 'BT': 'BUKIT TIMAH', 'CCK': 'CHOA CHU KANG', 'CL': 'CLEMENTI', 'CT': 'CENTRAL AREA',
    'GL': 'GEYLANG', 'HG': 'HOUGANG', 'JE': 'JURONG EAST', 'JW': 'JURONG WEST', 'KWN': 'KALLANG/WHAMPOA',
    'MP': 'MARINE PARADE', 'PG': 'PUNGGOL', 'PRC': 'PASIR RIS', 'QT': 'QUEENSTOWN', 'SB': 'SEMBAWANG',
    'SGN': 'SERANGOON', 'SK': 'SENGKANG', 'TAP': 'TAMPINES', 'TG': 'TENGAH', 'TP' : 'TOA PAYOH' ,
    'WL' : 'WOODLANDS' , 'YS' : 'YISHUN'
}

async def extract_raw_data_async(resource_id):
    data_gov_base_url = "https://data.gov.sg/api/action/datastore_search?resource_id="
    try:
        base_url = f"{data_gov_base_url}{resource_id}"
        check = requests.get(base_url)
        record_count = check.json()["result"]["total"]
    except KeyError:
        print(f"Invalid result response, check the url: {base_url}")
        return None
    except Exception as e:
        print(f"Failed to access the API at url: {base_url}")
        print(e)
        return None

    async with aiohttp.ClientSession() as session:
        async def fetch(url):
            async with session.get(url) as response:
                if response.status != 200:
                    response.raise_for_status()
                return await response.json()

        rows_per_iteration = min(math.ceil(record_count / 10), 5000)
        num_calls = math.ceil(record_count / rows_per_iteration)

        urls = [f"{base_url}&offset={i * rows_per_iteration}&limit={rows_per_iteration}" for i in range(num_calls)]
        tasks = [fetch(url) for url in urls]
        downloaded_records = await asyncio.gather(*tasks)

    print(num_calls)
    return [record for call in downloaded_records for record in call['result']['records']]


def extract_buildings():
    records = asyncio.run(extract_raw_data_async(datagov_name_and_id_buildings[1]))
    df = pd.DataFrame(records)
    df = df.drop(df.columns[0], axis=1)
    return df

def transform_buildings(buildings):
    # Run OneMap API to get postal code 
    buildings['postal_code'] = onemap_api_postal(buildings)

    # Convert to the correct formatting for Town
    buildings['bldg_contract_town'] = buildings['bldg_contract_town'].map(town_mapping)

    # rename town
    buildings = buildings.rename({'bldg_contract_town' : 'town'}, axis = 1)

    return buildings

def create_flats_sold(buildings):
    # Extracting required columns
    flat_type_df = pd.DataFrame()
    flat_type_df['postal_code'] = buildings['postal_code']
    
    # Extracting flat types and their respective sold counts
    renamed_flat_types = {'1room_sold': '1 ROOM', '2room_sold': '2 ROOM', '3room_sold': '3 ROOM',
                        '4room_sold': '4 ROOM', '5room_sold' : '5 ROOM', 'exec_sold' : 'EXECUTIVE', 
                        'multigen_sold' : 'MULTI-GENERATION'}

    # Creating new columns for each flat type and their sold counts
    for flat_type, renamed_flat_type in renamed_flat_types.items():
        flat_type_df[renamed_flat_type] = buildings[flat_type]
    
    # Reshaping the DataFrame to have flat_type as a column
    flat_type_df = pd.melt(flat_type_df, id_vars=['postal_code'], value_vars=list(renamed_flat_types.values()),
                            var_name='flat_type', value_name='flats_sold')
    
    return flat_type_df

def drop_building_columns(buildings):
    # Remove irrelevant columns
    retained_cols = ['postal_code', 'blk_no', 'street', 'max_floor_lvl', 'year_completed', 
                    'multistorey_carpark', 'precinct_pavilion', 'market_hawker']
    
    buildings = buildings[retained_cols]
    return buildings

def onemap_api_postal(data):
    async def fetch(session, blk_no, street):
        search_val = f"{blk_no} {street}"
        url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_val}&returnGeom=Y&getAddrDetails=Y&pageNum=1"

        async with session.get(url) as response:
            try:
                query = await response.json()
                query = query['results'][0]  
                postal = query['POSTAL']
                return postal
            except:
                print(f'API Query failed for {blk_no} , {street}')
                return None
            
    async def main():
        tasks = []
        async with aiohttp.ClientSession() as session:
            for index, row in data.iterrows():
                task = asyncio.create_task(fetch(session, row['blk_no'], row['street']))
                tasks.append(task)
            results = await asyncio.gather(*tasks)
        return results

    return asyncio.run(main())

In [86]:
buildings = extract_buildings()
buildings = transform_buildings(buildings)
flats_sold = create_flats_sold(buildings)
buildings = drop_building_columns(buildings)

10
API Query failed for 141A , SERANGOON NTH AVE 2
API Query failed for 4 , WOODLANDS ST 12
API Query failed for 5 , BANDA ST


KeyError: '1room_sold'

In [87]:
buildings

,postal_code,blk_no,street,max_floor_lvl,year_completed,multistorey_carpark,precinct_pavilion,market_hawker
0,190001,1,BEACH RD,16,1970,N,N,N
1,460001,1,BEDOK STH AVE 1,14,1975,N,N,N
2,080001,1,CANTONMENT RD,2,2010,N,N,N
3,461001,1,CHAI CHEE RD,15,1982,N,N,N
4,500001,1,CHANGI VILLAGE RD,4,1975,N,N,N
...,...,...,...,...,...,...,...,...
12821,531998,998A,BUANGKOK CRES,18,2018,N,N,N
12822,532998,998B,BUANGKOK CRES,17,2018,N,N,N
12823,530999,999,BUANGKOK CRES,2,2018,N,Y,N
12824,531999,999A,BUANGKOK CRES,18,2018,N,N,N


In [16]:
transactions = pd.read_csv('transactions2_data.csv', index_col=0).reset_index(drop = True)
buildings = pd.read_csv('buildings_data.csv', index_col=0).reset_index(drop =True)
schools = pd.read_csv('schools.csv')
medianrent = pd.read_csv('rent.csv')

In [17]:
def make_title(df):
    string_columns = df.select_dtypes(include=['object'])
    df[string_columns.columns] = string_columns.map(str.title)
    return df.reset_index(drop = True)

In [18]:
transactions.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [19]:
schools.head()

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,CO-ED SCHOOL,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
1,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,31 WOODLANDS CRESCENT,737916,63651733,63654596,63652774,na,Admiralty_SS@moe.edu.sg,ADMIRALTY MRT,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
3,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,751 YISHUN AVENUE 7,768928,67585384,na,67557778,na,aiss@moe.edu.sg,"CANBERRA MRT, YISHUN MRT",...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na


In [20]:
medianrent

,quarter,town,flat_type,median_rent
0,2005-Q2,ANG MO KIO,1-RM,na
1,2005-Q2,ANG MO KIO,2-RM,na
2,2005-Q2,ANG MO KIO,3-RM,800
3,2005-Q2,ANG MO KIO,4-RM,950
4,2005-Q2,ANG MO KIO,5-RM,-
...,...,...,...,...
11272,2023-Q1,YISHUN,2-RM,-
11273,2023-Q1,YISHUN,3-RM,2500
11274,2023-Q1,YISHUN,4-RM,3000
11275,2023-Q1,YISHUN,5-RM,3200


In [21]:
medianrent.head()

,quarter,town,flat_type,median_rent
0,2005-Q2,ANG MO KIO,1-RM,na
1,2005-Q2,ANG MO KIO,2-RM,na
2,2005-Q2,ANG MO KIO,3-RM,800
3,2005-Q2,ANG MO KIO,4-RM,950
4,2005-Q2,ANG MO KIO,5-RM,-


# Flat Type

In [22]:
medianrent['flat_type'].unique()

array(['1-RM', '2-RM', '3-RM', '4-RM', '5-RM', 'EXEC'], dtype=object)

In [23]:
transactions['flat_type'].unique()

array(['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', '1 ROOM',
       'MULTI-GENERATION'], dtype=object)

In [24]:
def load_flat_type(transactions):
    flat_type = pd.DataFrame(transactions['flat_type'].unique(), columns = ['flat_type'])
    return flat_type

load_flat_type(transactions)

,flat_type
0,2 ROOM
1,3 ROOM
2,4 ROOM
3,5 ROOM
4,EXECUTIVE
5,1 ROOM
6,MULTI-GENERATION


In [83]:
def onemap_api(data):
    async def fetch(session, blk_no, street):
        search_val = f"{blk_no} {street}"
        url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_val}&returnGeom=Y&getAddrDetails=Y&pageNum=1"

        async with session.get(url) as response:
            try:
                query = await response.json()
                query = query['results'][0]  
                postal = query['POSTAL']
                return postal
            except:
                print(f'API Query failed for {blk_no} , {street}')
                return None
            
    async def main():
        tasks = []
        async with aiohttp.ClientSession() as session:
            for index, row in data.iterrows():
                task = asyncio.create_task(fetch(session, row['blk_no'], row['street']))
                tasks.append(task)
            results = await asyncio.gather(*tasks)
        return results

    return asyncio.run(main())

# Buildings

In [130]:
buildings = pd.read_csv('buildings.csv', index_col = 0).reset_index(drop = True)
buildings

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,...,138,1,2,0,0,0,0,0,0,0
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,...,204,0,2,0,0,0,0,0,0,0
2,1,CANTONMENT RD,2,2010,N,Y,N,N,N,N,...,0,0,0,0,0,0,0,0,0,0
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,...,0,10,92,0,0,0,0,0,0,0
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,...,54,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821,998A,BUANGKOK CRES,18,2018,Y,N,N,N,N,N,...,33,50,0,0,0,0,0,0,0,0
12822,998B,BUANGKOK CRES,17,2018,Y,N,N,N,N,N,...,31,47,0,0,0,0,0,0,0,0
12823,999,BUANGKOK CRES,2,2018,N,N,N,Y,N,Y,...,0,0,0,0,0,0,0,0,0,0
12824,999A,BUANGKOK CRES,18,2018,Y,N,N,N,N,N,...,0,0,0,0,0,0,136,170,0,0


In [3]:
import math
def extract_raw_data():
    return asyncio.run(extract_raw_data_async())

async def extract_raw_data_async():
    data_gov_base_url = "https://data.gov.sg/api/action/datastore_search?resource_id="
    # headers = {"User-Agent": "Mozilla/5.0"}

    datagov_name_and_ids = [
        ("buildings", "d_17f5382f26140b1fdae0ba2ef6239d2f"),
        ("transactions1_data", "d_ebc5ab87086db484f88045b47411ebc5"),
        ("transactions2_data", "d_43f493c6c50d54243cc1eab0df142d6a"),
        ("transactions3_data", "d_2d5ff9ea31397b66239f245f57751537"),
        ("transactions4_data", "d_ea9ed51da2787afaf8e51f827c304208"),
        ("transactions5_data", "d_8b84c4ee58e3cfc0ece0d773c8ca6abc"),
        ("median_rent", "d_23000a00c52996c55106084ed0339566"),
        ("schools", "d_688b934f82c1059ed0a6993d2a829089")
    ]

    async def datagovsg_id_to_csv_async(resource_id, filename, row_max_limit=5000, iterations=10):
        try:
            base_url = f"{data_gov_base_url}{resource_id}"
            check = requests.get(base_url)
            record_count = check.json()["result"]["total"]
        except KeyError:
            print(f"Invalid result response, check the url: {base_url}")
            return None
        except Exception as e:
            print(f"Failed to access the API at url: {base_url}")
            print(e)
            return None
        
        async def fetch(s, url):
            async with s.get(url) as result:
                if result.status != 200:
                    result.raise_for_status()
                output = await result.json()
                try:
                    output = output["result"]["records"]
                    return output
                except:
                    print(f"Error downloading at at url: {url}")
                    print(output)

        async def fetch_all(s, urls):
            tasks = []
            for url in urls:
                task = asyncio.create_task(fetch(s, url))
                tasks.append(task)
            completed = await asyncio.gather(*tasks)
            result = []
            for complete in completed:
                result.extend(complete)
            return result

        rows_per_iteration = min(math.ceil(record_count / iterations), row_max_limit)
        num_calls = math.ceil(record_count / rows_per_iteration)

        urls = [f"{base_url}&offset={i * rows_per_iteration}&limit={rows_per_iteration}" for i in range(0, num_calls)]
        async with aiohttp.ClientSession() as s:
            downloaded_records = await fetch_all(s, urls)
        
        if len(downloaded_records) != record_count:
            print(f"Dataset {resource_id} ({filename}) did not download completely.")
            print(f"Expected row counts: {record_count}")
            print(f"Downloaded row counts: {len(downloaded_records)}")

        pd.DataFrame.from_records(downloaded_records).to_csv(f"{filename}.csv", index=False)
        return 
        
    tasks = []
    for filename, dataset_id in datagov_name_and_ids:
        task = asyncio.create_task(datagovsg_id_to_csv_async(dataset_id, filename))
        tasks.append(task)
    await asyncio.gather(*tasks)

    return [filename for filename, _ in datagov_name_and_ids]

extract_raw_data()

['buildings',
 'transactions1_data',
 'transactions2_data',
 'transactions3_data',
 'transactions4_data',
 'transactions5_data',
 'median_rent',
 'schools']

: 

In [85]:
def transform_buildings(buildings):
    # Run OneMap API to get postal code
    buildings['postal_code'] = onemap_api(buildings)
    return buildings

buildings = transform_buildings(buildings)

KeyboardInterrupt: 

In [127]:
def transform_flats_sold(buildings):
    # Extracting required columns
    flat_type_df = pd.DataFrame()
    flat_type_df['postal_code'] = buildings['postal_code']
    
    # Extracting flat types and their respective sold counts
    renamed_flat_types = {'1room_sold': '1 ROOM', '2room_sold': '2 ROOM', '3room_sold': '3 ROOM',
                          '4room_sold': '4 ROOM', '5room_sold' : '5 ROOM', 'exec_sold' : 'EXECUTIVE', 
                          'multigen_sold' : 'MULTI-GENERATION'}

    # Creating new columns for each flat type and their sold counts
    for flat_type, renamed_flat_type in renamed_flat_types.items():
        flat_type_df[renamed_flat_type] = buildings[flat_type]
    
    # Reshaping the DataFrame to have flat_type as a column
    flat_type_df = pd.melt(flat_type_df, id_vars=['postal_code'], value_vars=list(renamed_flat_types.values()),
                            var_name='flat_type', value_name='flats_sold')
    
    return flat_type_df

flats_sold = transform_flats_sold(buildings)


In [128]:
flats_sold

,postal_code,flat_type,flats_sold
0,190001,1 ROOM,0
1,460001,1 ROOM,0
2,080001,1 ROOM,0
3,461001,1 ROOM,0
4,500001,1 ROOM,0
...,...,...,...
89777,531998,MULTI-GENERATION,0
89778,532998,MULTI-GENERATION,0
89779,530999,MULTI-GENERATION,0
89780,531999,MULTI-GENERATION,0


# Town

In [ ]:
town_mapping = {
        'AMK': 'ANG MO KIO', 'BB': 'BUKIT BATOK', 'BD': 'BEDOK', 'BH': 'BISHAN', 'BM': 'BUKIT MERAH',
        'BP': 'BUKIT PANJANG', 'BT': 'BUKIT TIMAH', 'CCK': 'CHOA CHU KANG', 'CL': 'CLEMENTI', 'CT': 'CENTRAL AREA',
        'GL': 'GEYLANG', 'HG': 'HOUGANG', 'JE': 'JURONG EAST', 'JW': 'JURONG WEST', 'KWN': 'KALLANG/WHAMPOA',
        'MP': 'MARINE PARADE', 'PG': 'PUNGGOL', 'PRC': 'PASIR RIS', 'QT': 'QUEENSTOWN', 'SB': 'SEMBAWANG',
        'SGN': 'SERANGOON', 'SK': 'SENGKANG', 'TAP': 'TAMPINES', 'TG': 'TENGAH', 'TP' : 'TOA PAYOH' ,
        'WL' : 'WOODLANDS' , 'YS' : 'YISHUN'
}

In [ ]:
buildings['bldg_contract_town'].unique()

array(['KWN', 'BD', 'CT', 'PRC', 'BM', 'QT', 'GL', 'HG', 'SGN', 'TP',
       'MP', 'WL', 'PG', 'BT', 'TAP', 'CCK', 'AMK', 'BH', 'BB', 'SB',
       'CL', 'BP', 'JE', 'SK', 'YS', 'JW', 'TG'], dtype=object)

In [ ]:
def load_town(town_mapping):
    return pd.DataFrame(list(town_mapping.values()), columns = ['town'])

load_town(town_mapping)

,town
0,ANG MO KIO
1,BUKIT BATOK
2,BEDOK
3,BISHAN
4,BUKIT MERAH
5,BUKIT PANJANG
6,BUKIT TIMAH
7,CHOA CHU KANG
8,CLEMENTI
9,CENTRAL AREA


In [ ]:
def rename_buildings_town(data):
    data = data.copy(deep = True)

    data['bldg_contract_town'] = data['bldg_contract_town'].map(town_mapping)
    return data

rename_buildings_town(buildings)['bldg_contract_town']

0        KALLANG/WHAMPOA
1                  BEDOK
2           CENTRAL AREA
3                  BEDOK
4              PASIR RIS
              ...       
12821            HOUGANG
12822            HOUGANG
12823            HOUGANG
12824            HOUGANG
12825            HOUGANG
Name: bldg_contract_town, Length: 12826, dtype: object

# Transform buildings v2

In [ ]:
def drop_buildings_columns(buildings):
    retained_cols = ['postal_code', 'blk_no', 'street', 'max_floor_lvl', 'year_completed', 'multistorey_carpark', 'precinct_pavilion', 'market_hawker']
    buildings_new = buildings[retained_cols]
    return buildings_new

drop_buildings_columns(buildings)

,postal_code,blk_no,street,max_floor_lvl,year_completed,multistorey_carpark,precinct_pavilion,market_hawker
0,190001,1,BEACH RD,16,1970,N,N,N
1,460001,1,BEDOK STH AVE 1,14,1975,N,N,N
2,080001,1,CANTONMENT RD,2,2010,N,N,N
3,461001,1,CHAI CHEE RD,15,1982,N,N,N
4,500001,1,CHANGI VILLAGE RD,4,1975,N,N,N
...,...,...,...,...,...,...,...,...
12821,531998,998A,BUANGKOK CRES,18,2018,N,N,N
12822,532998,998B,BUANGKOK CRES,17,2018,N,N,N
12823,530999,999,BUANGKOK CRES,2,2018,N,Y,N
12824,531999,999A,BUANGKOK CRES,18,2018,N,N,N


# Median Rent Prices

In [ ]:
medianrent.head()

,quarter,town,flat_type,median_rent
0,2005-Q2,ANG MO KIO,1-RM,na
1,2005-Q2,ANG MO KIO,2-RM,na
2,2005-Q2,ANG MO KIO,3-RM,800
3,2005-Q2,ANG MO KIO,4-RM,950
4,2005-Q2,ANG MO KIO,5-RM,-


In [ ]:
medianrent = pd.read_csv('rent.csv')

def transform_median_rent(rent):
    renamed_flat_types = {'1-RM': '1 ROOM', '2-RM': '2 ROOM', '3-RM': '3 ROOM',
                          '4-RM': '4 ROOM', '5-RM' : '5 ROOM', 'EXEC' : 'EXECUTIVE'}   
                          
    # Splitting quarter column into year and quarter columns
    rent[['year', 'quarter']] = rent['quarter'].str.split('-', expand=True)
    rent['year'] = pd.to_numeric(rent['year'])
    
    # remove unnecessary trailing spaces
    rent = rent.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

    # replace na values and - values with NaN
    rent.replace(r'^\s*na\s*$', np.nan, regex = True, inplace=True)
    rent.replace(r'^\s*-\s*$', np.nan, regex = True, inplace=True)
    
    # convert flat type to consistent format
    rent['flat_type'] = rent['flat_type'].map(renamed_flat_types)

    # reorder the columns
    rent = rent[['year', 'quarter', 'town', 'flat_type', 'median_rent']]
    return rent

transform_median_rent(medianrent)

,year,quarter,town,flat_type,median_rent
0,2005,Q2,ANG MO KIO,1 ROOM,NaN
1,2005,Q2,ANG MO KIO,2 ROOM,NaN
2,2005,Q2,ANG MO KIO,3 ROOM,800
3,2005,Q2,ANG MO KIO,4 ROOM,950
4,2005,Q2,ANG MO KIO,5 ROOM,NaN
...,...,...,...,...,...
11272,2023,Q1,YISHUN,2 ROOM,NaN
11273,2023,Q1,YISHUN,3 ROOM,2500
11274,2023,Q1,YISHUN,4 ROOM,3000
11275,2023,Q1,YISHUN,5 ROOM,3200


In [ ]:
medianrent['flat_type'].unique()

array(['1-RM', '2-RM', '3-RM', '4-RM', '5-RM', 'EXEC'], dtype=object)

# Schools

In [ ]:
schools = pd.read_csv('schools.csv')

In [ ]:
schools.head()

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,...,CO-ED SCHOOL,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
1,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,31 WOODLANDS CRESCENT,737916,63651733,63654596,63652774,na,Admiralty_SS@moe.edu.sg,ADMIRALTY MRT,...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
3,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,751 YISHUN AVENUE 7,768928,67585384,na,67557778,na,aiss@moe.edu.sg,"CANBERRA MRT, YISHUN MRT",...,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,...,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na


In [ ]:
schools['mrt_desc'].unique()

array(['Admiralty Station', 'ADMIRALTY MRT', 'Yishun',
       'CANBERRA MRT, YISHUN MRT', 'Bishan MRT',
       'Redhill Station  Tiong Bahru Station',
       'MRT : NE16-Sengkang LRT : SW7-TongKang',
       'Yio Chu Kang MRT Station', 'ANG MO KIO MRT, YIO CHU KANG MRT',
       'Ang Mo Kio MRT Station', 'ANG MO KIO MRT', 'TANAH MERAH MRT',
       'Buona Vista', 'NEWTON MRT, STEVENS MRT',
       'DOVER MRT, BUONA VISTA MRT, ONE-NORTH MRT', 'Newton MRT Station',
       'Tampines and Simei Stations', 'CASHEW MRT',
       'DOWNTOWN LINE 2, CASHEW MRT', 'BARTLEY MRT, TAI SENG MRT',
       'Bangkit LRT, Fajar LRT Station', 'BRADDELL MRT, TOA PAYOH MRT',
       'Bedok MRT Station, Tanah Merah MRT Station',
       'BEDOK MRT, BEDOK RESERVOIR MRT', 'Boon Keng, Kallang',
       'BOON KENG MRT', 'Telok Blangah, Tiong Bahru & Redhill',
       'Lakeside & Boon Lay', 'BOON LAY MRT',
       'HOUGANG MRT, SERANGOON MRT, SENGKANG MRT', 'DAKOTA MRT',
       'BUKIT BATOK MRT', 'REDHILL MRT', 'CHOA CHU KAN

In [ ]:
schools = pd.read_csv('schools.csv')
import re

def transform_schools(schools):
    def clean_and_format_town_name(text):
        # Remove stopwords and make format better
        text = text.title()
        cleaned_text = re.sub(r'\b(?:Station|Mrt|Lrt|Line|\([^)]*\))\b', '', text)
        return cleaned_text.strip()
    
    schools['mrt_desc'] = schools['mrt_desc'].apply(lambda x: clean_and_format_town_name(x))

    retained_cols = ['school_name', 'postal_code', 'mrt_desc', 'mainlevel_code', 
                     'nature_code', 'type_code', 'sap_ind', 'autonomous_ind', 'gifted_ind', 'ip_ind']
    
    return schools[retained_cols].rename(columns={'mrt_desc': 'closest_mrt'})

transform_schools(schools)

,school_name,postal_code,closest_mrt,mainlevel_code,nature_code,type_code,sap_ind,autonomous_ind,gifted_ind,ip_ind
0,ADMIRALTY PRIMARY SCHOOL,738907,Admiralty,PRIMARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No
1,ADMIRALTY SECONDARY SCHOOL,737916,Admiralty,SECONDARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No
2,AHMAD IBRAHIM PRIMARY SCHOOL,768643,Yishun,PRIMARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No
3,AHMAD IBRAHIM SECONDARY SCHOOL,768928,"Canberra , Yishun",SECONDARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No
4,AI TONG SCHOOL,579646,Bishan,PRIMARY,CO-ED SCHOOL,GOVERNMENT-AIDED SCH,Yes,No,No,No
...,...,...,...,...,...,...,...,...,...,...
341,ZHANGDE PRIMARY SCHOOL,169485,Tiong Bahru,PRIMARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No
342,ZHENGHUA PRIMARY SCHOOL,679002,: Fajar,PRIMARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No
343,ZHENGHUA SECONDARY SCHOOL,677741,Jelapang,SECONDARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No
344,ZHONGHUA PRIMARY SCHOOL,556095,"Ang Mo Kio , Bishan , Serangoon",PRIMARY,CO-ED SCHOOL,GOVERNMENT SCHOOL,No,No,No,No


# Transactions

In [ ]:
transactions = pd.read_csv('transactions2_data.csv', index_col=0).reset_index()
transactions.rename({'_id' : 'transaction_id'} , axis = 1)


,transaction_id,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,1,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,3,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,4,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,5,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
175836,175837,2024-03,YISHUN,EXECUTIVE,392,YISHUN AVE 6,07 TO 09,146.0,Maisonette,1988,63 years 04 months,830000.0
175837,175838,2024-03,YISHUN,EXECUTIVE,406,YISHUN AVE 6,10 TO 12,142.0,Apartment,1988,63 years 06 months,815000.0
175838,175839,2024-03,YISHUN,EXECUTIVE,326,YISHUN RING RD,10 TO 12,146.0,Maisonette,1988,63 years 05 months,820000.0
175839,175840,2024-03,YISHUN,EXECUTIVE,360,YISHUN RING RD,04 TO 06,146.0,Maisonette,1988,63 years 05 months,860000.0


In [92]:
def read_transactions():
    trans1 = pd.read_csv('transactions1_data.csv', index_col=0).reset_index(drop = True)
    trans2 = pd.read_csv('transactions2_data.csv', index_col=0).reset_index(drop = True)
    trans3 = pd.read_csv('transactions3_data.csv', index_col=0).reset_index(drop = True)
    trans4 = pd.read_csv('transactions4_data.csv', index_col=0).reset_index(drop = True)
    trans5 = pd.read_csv('transactions5_data.csv', index_col=0).reset_index(drop = True)

    merged_transactions = pd.concat([trans1, trans2, trans3, trans4, trans5], ignore_index=True).reset_index()
    merged_transactions = merged_transactions.rename({'index': 'transaction_id'}, axis = 1)
    return merged_transactions

read_transactions()

,transaction_id,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
922038,922038,2024-03,YISHUN,EXECUTIVE,392,YISHUN AVE 6,07 TO 09,146.0,Maisonette,1988,830000.0,63 years 04 months
922039,922039,2024-03,YISHUN,EXECUTIVE,406,YISHUN AVE 6,10 TO 12,142.0,Apartment,1988,815000.0,63 years 06 months
922040,922040,2024-03,YISHUN,EXECUTIVE,326,YISHUN RING RD,10 TO 12,146.0,Maisonette,1988,820000.0,63 years 05 months
922041,922041,2024-03,YISHUN,EXECUTIVE,360,YISHUN RING RD,04 TO 06,146.0,Maisonette,1988,860000.0,63 years 05 months


In [113]:
def transform_transactions(transactions):
    # Drop  the remaining lease columns
    transactions = transactions.drop(['remaining_lease'], axis = 1)

    # Apply formatting to flat_model
    transactions['flat_model'] = transactions['flat_model'].str.upper()

    # Convert 'MULTI GENERATION' to 'MULTI-GENERATION'
    transactions['flat_type'] = transactions['flat_type'].str.replace('MULTI GENERATION', 'MULTI-GENERATION')
    
    # rename lease_commence_date
    transactions = transactions.rename({'lease_commence_date' : 'lease_commence_year'}, axis = 1)
    return transactions

transactions = transform_transactions(read_transactions())

In [114]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 922043 entries, 0 to 922042
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   transaction_id       922043 non-null  int64  
 1   month                922043 non-null  object 
 2   town                 922043 non-null  object 
 3   flat_type            922043 non-null  object 
 4   block                922043 non-null  object 
 5   street_name          922043 non-null  object 
 6   storey_range         922043 non-null  object 
 7   floor_area_sqm       922043 non-null  float64
 8   flat_model           922043 non-null  object 
 9   lease_commence_year  922043 non-null  int64  
 10  resale_price         922043 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 77.4+ MB


In [108]:
transactions.describe()

,transaction_id,floor_area_sqm,lease_commence_date,resale_price
count,922043.000000,922043.000000,922043.000000,9.220430e+05
mean,461021.000000,95.694553,1988.221797,3.192139e+05
std,266171.031465,25.834902,10.607035,1.690929e+05
min,0.000000,28.000000,1966.000000,5.000000e+03
25%,230510.500000,73.000000,1981.000000,1.925000e+05
50%,461021.000000,93.000000,1986.000000,2.950000e+05
75%,691531.500000,113.000000,1996.000000,4.150000e+05
max,922042.000000,307.000000,2022.000000,1.568888e+06


In [109]:
transactions['flat_model'].unique()

array(['IMPROVED', 'NEW GENERATION', 'MODEL A', 'STANDARD', 'SIMPLIFIED',
       'MODEL A-MAISONETTE', 'APARTMENT', 'MAISONETTE', 'TERRACE',
       '2-ROOM', 'IMPROVED-MAISONETTE', 'MULTI GENERATION',
       'PREMIUM APARTMENT', 'ADJOINED FLAT', 'PREMIUM MAISONETTE',
       'MODEL A2', 'DBSS', 'TYPE S1', 'TYPE S2', 'PREMIUM APARTMENT LOFT',
       '3GEN'], dtype=object)

In [110]:
transactions['flat_type'].unique()

array(['1 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE',
       'MULTI-GENERATION'], dtype=object)

In [100]:
transactions['lease_commence_date'].unique()

array([1977, 1976, 1978, 1979, 1984, 1980, 1985, 1981, 1982, 1986, 1972,
       1983, 1973, 1969, 1975, 1971, 1974, 1967, 1970, 1968, 1988, 1987,
       1989, 1990, 1992, 1993, 1994, 1991, 1995, 1996, 1997, 1998, 1999,
       2000, 2001, 1966, 2002, 2006, 2003, 2005, 2004, 2008, 2007, 2009,
       2010, 2012, 2011, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2022,
       2020])

In [101]:
transactions['floor_area_sqm'].unique()

array([ 31. ,  73. ,  67. ,  82. ,  74. ,  88. ,  89. ,  83. ,  68. ,
        75. ,  81. ,  91. ,  92. ,  97. ,  90. ,  98. ,  99. , 100. ,
        93. , 103. , 119. , 120. , 118. , 121. , 135. , 117. ,  45. ,
        65. ,  59. ,  70. ,  76. ,  84. , 104. , 105. , 125. , 132. ,
       139. , 123. , 143. , 151. ,  69. , 106. , 107. , 116. , 149. ,
       141. , 146. , 148. , 145. , 154. , 150. ,  29. ,  51. ,  61. ,
        63. ,  64. ,  72. ,  58. ,  66. ,  60. ,  53. ,  54. ,  56. ,
        77. , 133. , 131. , 115. ,  43. ,  38. ,  41. ,  85. , 111. ,
       101. , 112. , 137. , 127. , 147. , 163. ,  50. ,  40. ,  60.3,
        62. ,  55. ,  57. ,  52. ,  63.1, 102. ,  83.1, 126. , 140. ,
       142. ,  71. , 108. , 144. ,  96. , 114. , 157. , 152. , 155. ,
        87. , 109. , 110. ,  94. , 134. , 122. , 128. ,  78. ,  46. ,
        42. ,  49. ,  47. ,  86. , 156. ,  79. ,  80. , 124. ,  28. ,
       113. ,  95. , 160. , 136. ,  48. , 138. , 161. ,  39. , 130. ,
       159. , 206. ,

# MRT Station

In [1]:
def extract_mrt_stations():
    def onemap_api_mrt_stations():
        # Making simple request query to get totalNumPages
        search_val = "MRT Station"
        url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_val}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
        response = requests.request("GET", url)
        num_pages = response.json()['totalNumPages']

        # Async Fetch function
        async def fetch(session, page_num):
            url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_val}&returnGeom=Y&getAddrDetails=Y&pageNum={page_num}"
            async with session.get(url) as response:
                try:
                    query = await response.json()
                    query = query['results']
                    return query
                except:
                    return None
        
        # Async main function to fetch all records
        async def main():
            tasks = []
            page_num = 1 # Starts with page 1 of query
            async with aiohttp.ClientSession() as session:
                while page_num <= num_pages:
                    task = asyncio.create_task(fetch(session, page_num))
                    tasks.append(task)
                    page_num += 1
                results = await asyncio.gather(*tasks)
            return results

        return asyncio.run(main())
  
    mrt_stations_query = onemap_api_mrt_stations()

    # Flatten nested list
    stations = [(station['BUILDING'], station['POSTAL']) for sublist in mrt_stations_query for station in sublist if '(' in station['BUILDING'] and ')' in station['BUILDING']]
    
    # Convert stations into dataframe and remove duplicates
    stations = pd.DataFrame(stations, columns = ['station_name', 'postal_code']).drop_duplicates(subset = ['postal_code'])
    return stations

In [4]:
stations = extract_mrt_stations()

In [7]:
stations.to_csv('mrt_stations.csv', index = 0)

In [123]:
list(stations['station_name'])

['LENTOR MRT STATION (TE5)',
 'SOMERSET MRT STATION (NS23)',
 'CALDECOTT MRT STATION (TE9)',
 'CASHEW MRT STATION (DT2)',
 'UPPER THOMSON MRT STATION (TE8)',
 'MARINA BAY MRT STATION (NS27)',
 'MARINA BAY MRT STATION (CE2)',
 'SHENTON WAY MRT STATION (TE19)',
 'MARINA BAY MRT STATION (TE20)',
 'MARINA SOUTH PIER MRT STATION (NS28)',
 'BAYFRONT MRT STATION (DT16 / CE1)',
 'DOWNTOWN MRT STATION (DT17)',
 'GARDENS BY THE BAY MRT STATION (TE22)',
 'MARINA SOUTH MRT STATION (TE21)',
 'EXPO MRT STATION (DT35)',
 'PROMENADE MRT STATION (CC4)',
 'KING ALBERT PARK MRT STATION (DT6)',
 'TAMPINES WEST MRT STATION (DT31)',
 'NAPIER MRT STATION (TE12)',
 'TANJONG KATONG MRT STATION (TE25)',
 'TANJONG RHU MRT STATION (TE23)',
 'SUNGEI BEDOK MRT STATION (TE31 / DT37)',
 'STADIUM MRT STATION (CC6)',
 'NICOLL HIGHWAY MRT STATION (CC5)',
 'PROMENADE MRT STATION (DT15)',
 'TELOK AYER MRT STATION (DT18)',
 'RAFFLES PLACE MRT STATION (EW14 / NS26)',
 'KALLANG MRT STATION (EW10)',
 'SENGKANG MRT STATION (NE

In [18]:
import pandas as pd
import numpy as np

In [5]:
buildings = pd.read_csv("buildings.csv")
flats_sold = pd.read_csv("flats_sold.csv")



int64
object
object
int64
int64
object
object
object


In [20]:
dtype_mapper = dict(zip(flats_sold.columns, flats_sold.dtypes))
dtype_mapper

{'postal_code': dtype('int64'),
 'flat_type': dtype('O'),
 'flats_sold': dtype('int64')}

In [21]:
dtype_mapper["postal_code"] = np.dtype('O')

In [22]:
flats_sold2 = pd.read_csv("flats_sold.csv", dtype=dtype_mapper)